In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import tensorflow as tf
import albumentations
import pandas as pd
import numpy as np
import shutil
import os
from sklearn.model_selection import KFold, StratifiedKFold

In [ ]:

import numpy as np
import pandas as pd
train = pd.read_csv('../input/plant-pathology-2021-fgvc8/train.csv')
train.head()

In [ ]:
class CFG():
    img_size = 299
    seed = 42
    classes = ['complex', 'frog_eye_leaf_spot', 'healthy', 'powdery_mildew', 'rust', 'scab']
    batch_size=16

In [ ]:
# train = pd.read_csv('../input/plant-pathology-2021-fgvc8/train.csv')

#train,test = train[0:int(len(paths)*0.8)],train[int(len(paths)*0.8):]
train = pd.DataFrame(train,columns = ['image','labels'])
train['labels'].value_counts()

In [ ]:
train_dir='../input/plant-pathology-2021-fgvc8/train_images'
# submission_dir='../input/plant-pathology-2021-fgvc8/test_images'

In [ ]:
test_img = '../input/plant-pathology-2021-fgvc8/test_images'
submission = pd.read_csv('../input/plant-pathology-2021-fgvc8/sample_submission.csv')
submission.head()

In [ ]:
submission['labels'] = submission['labels'].apply(lambda s: s.split(' '))
submission.head()

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer
mb = MultiLabelBinarizer().fit(train.labels.apply(lambda x : x.split()))
labels = pd.DataFrame(mb.transform(train.labels.apply(lambda x : x.split())), columns = mb.classes_)
new_train = pd.concat([train['image'], labels], axis=1)
new_train.head()

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import Xception
from keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.preprocessing import MultiLabelBinarizer

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
import cv2
import numpy as np
def get_cut_image(image):
    img = cv2.GaussianBlur(image,(3,3),0)
    copy = np.uint8(img)
    canny = cv2.Canny(copy, 145, 165)
    box = np.argwhere(canny>0)
    y1,x1 = box.min(axis=0)
    y2,x2 = box.max(axis=0)
    cut_img = img[y1:y2, x1:x2]
    cut_img = cv2.resize(cut_img, (CFG.img_size,CFG.img_size))
    cut_img = cut_img.astype("float32")*(1.)/255
    return np.array(cut_img)
datagen = ImageDataGenerator(
    rescale = 1/255.,
    rotation_range = 15,
    width_shift_range=0.2, 
    height_shift_range=0.2,
    zoom_range = 0.2,
    shear_range=0.3, #（以弧度逆时针方向剪切角度）。
    horizontal_flip=True,
    vertical_flip=True,
    validation_split= 0.2,
    preprocessing_function = get_cut_image)

In [ ]:
test_generator = datagen.flow_from_dataframe(
                  submission,
                  directory = test_img,
                  x_col = 'image',
                  y_col = 'labels',
                  class_mode = "categorical",
                  batch_size=4,
                  target_size = (CFG.img_size,CFG.img_size),
                  color_mode="rgb",
                  shuffle = False,
                  seed = CFG.seed,
                  preprocessing_function = get_cut_image
)

In [ ]:
import keras
def load_model():
    model_pre = keras.models.load_model("../input/0720-cv-n-2/0720_Xception_CV_N_2.h5")
    return model_pre

In [ ]:
model = load_model()

In [ ]:
# model = load_model()
pred = model.predict(test_generator)

In [ ]:
perdict = (pred>0.33)
n_label = ['complex', 'frog_eye_leaf_spot', 'healthy', 'powdery_mildew', 'rust', 'scab']
answer = []

for i in range(perdict.shape[0]):
    temp = []
    for j, k in enumerate(n_label):
        if perdict[i, j]:
            temp.append(k)
    answer.append(temp)
    
answer = [' '.join(n) for n in answer]

In [ ]:
np.around(pred, decimals=3, out=None)

In [ ]:
submission['labels'] = np.array(answer)
submission

In [ ]:
submission.to_csv('submission.csv', index=False)